2022/2/8

This code is for extracting MFA values corresponding to each layer

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import cv2
import copy
import math
import itertools
import numpy.ma as ma
from tqdm import tqdm
from skimage.filters import threshold_otsu, gaussian
from skimage import measure
from skimage.draw import line
from scipy.optimize import curve_fit
from matplotlib import patches
from skimage.morphology import skeletonize, extrema
from skimage.measure import label
from skimage.feature import hessian_matrix, hessian_matrix_eigvals

In [2]:
#set function for edge detection
def detect_ridges(gray, sigma=1.0):
    H_elems = hessian_matrix(gray, sigma=sigma, order='rc')
    maxima_ridges, minima_ridges = hessian_matrix_eigvals(H_elems)
    return maxima_ridges, minima_ridges

def Lorentz_func(x, a, x0, sigma):
    return a *(sigma**2/(sigma**2+(x-x0)**2))

def gauss_func(x, a, mu, sigma):
    return a*np.exp(-(x-mu)**2/(2*sigma**2))


plt.rcParams["font.family"]="Times New Roman"
plt.rcParams["mathtext.fontset"]="stix"

In [3]:
#set path
current_path=os.getcwd()
npz_path=os.path.join(current_path, "npz_file")
save_path=os.path.join(current_path, "fig_save", "test")

if os.path.exists(save_path)==False:
    os.makedirs(save_path)

In [4]:
#load file
npz_file=os.listdir(npz_path)
npz_file_mfa=[i for i in npz_file if "matching_results_x40" in i]
npz_file_radialfile=[i for i in npz_file if "radial_file_result" in i]
npz_file_anatomy=[i for i in npz_file if "anatomical_parameters_x40" in i]

#load npz
#MFA
npz_mfa=np.load(os.path.join(npz_path, npz_file_mfa[0]))
height, width=npz_mfa["ret_im"].shape
MFA_im=npz_mfa["MFA_im"]
angle_im=npz_mfa["angle_im"]
flm_b_im=npz_mfa["flm_b"]
boundary_im=npz_mfa["boundary_im"]

#radial_file
npz_radial_file=np.load(os.path.join(npz_path, npz_file_radialfile[0]), allow_pickle=True)
radial_file_result_list=npz_radial_file["radial_file_result_list"]

#label map
npz_anatomy=np.load(os.path.join(npz_path, npz_file_anatomy[0]))
label_map=npz_anatomy["label_map"]
label_map_resize=npz_anatomy["nuclei_trim"]

In [5]:
#get backgorund
im=copy.deepcopy(MFA_im)
flm_copy=flm_b_im
threshold = threshold_otsu(flm_copy)
flm_binary=np.where(flm_copy<threshold, 255, 0) #255: lumen, 0: cellwall
flm_binary=np.uint8(flm_binary)
cell_index=np.where(flm_binary==0)

#background region is replaced by inf
im_bg_sub_1=np.where(flm_binary==0, im, np.inf)
im_bg_sub_2=np.where(flm_binary==0, im, 0)
im_ridge_detection=np.where(flm_binary==0, im, 0)

In [6]:
#get edge enhanced image
im_max, im_min=detect_ridges(im_ridge_detection, sigma=2)

In [7]:
#use im_min for extracting each layer
#thresholding
thres_min=threshold_otsu(im_min)
thres_max=threshold_otsu(im_max)

#binarization
binary_min=np.uint8(np.where(im_min>thres_min, 255, 0))
binary_max=np.uint8(np.where(im_max>thres_max, 255, 0))

#morphological operations
erosion = cv2.erode(np.uint8(binary_min), (7, 7), iterations = 5)
dilation = cv2.dilate(np.uint8(binary_min), (7, 7), iterations = 3)
erosion = np.uint8(np.where(erosion<255, 0, 255))
dilation = np.uint8(np.where(dilation<255, 0, 255))


In [8]:
#S1+P, S3 detetction
minor_layers=np.where(erosion==0, MFA_im, np.inf)

#S2 detetction
S2_layer=np.where(dilation==255, im_bg_sub_1, np.inf)

In [9]:
iteration_number=1
dst_thres=5
inner_parts=np.zeros((height, width))

target_map_1=np.uint8(np.where(flm_binary==255, 0, 255))
target_map_2=np.uint8(np.where(S2_layer==np.inf, 0, 255))
target_map_2=target_map_2+flm_binary

#calculate distance map of target_map_2 beforehand
dst_2 = cv2.distanceTransform(np.uint8(target_map_2), cv2.DIST_C, 3)

#set count
count=0

#while loop
while(True):
    #calculate distance map of target_map_1 
    dst_1 = cv2.distanceTransform(np.uint8(target_map_1), cv2.DIST_C, 3)

    #get inner area between S3 and lumen
    #Condition 1: inner part is defined as the pixels attached to lumen pixel
    #Condition 2: inner part is close to S3 wall.
    #To distinguish lumen side or S1 side, distances from both lumen and S3 are considered. 
    inner_part=np.where((dst_1==1) & (dst_2 < dst_thres), 255, 0)
    #inner_part=np.where((dst_1==1) & (dst_2==1), 255, 0)

    #sum up temporary result
    inner_parts+=inner_part
    
    #update target_map_1 for the next iteration
    #nya=copy.deepcopy(target_map_1)
    index=np.where(inner_part==255)
    target_map_1[index]=0

    #print(np.unique(np.abs(target_map_1-nya)))
    """
    if count==0:
        target_map_1=target_map_1*((inner_part==0)*1)

    else:
        target_map_prev=copy.deepcopy(target_map_1)
        target_map_1=target_map_1*((inner_part==0)*1)

        #if any update doesnt occur, break loop
        if len(np.unique(np.abs(target_map_1-target_map_prev)))==1:
            break
    """
    #if counter reaches iteration number you set, break loop
    if count==iteration_number:
        break
   

    print(count)
    #add count
    count+=1

0


The other minor noises are omitted by connected components.

In [10]:
S2_layer_correct=np.where(inner_parts>0, np.inf, S2_layer)
S2_layer_correct=np.where(minor_layers!=np.inf, np.inf, S2_layer_correct)
S2_layer_correct=np.where(S2_layer_correct==np.inf, 0, S2_layer_correct)
S2_layer_correct_binary=np.where(S2_layer_correct>0, 255, 0)

retval, labels, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(S2_layer_correct_binary))

#omit small noise by area
label_unique=np.unique(labels)
labels_select=[i for i in label_unique if stats[i][4]>100]
labels=labels.flatten()
pos=[list(np.where(labels==labels_select[i])[0]) for i in tqdm(range(len(labels_select)))]
pos=list(itertools.chain(*pos))

100%|████████████████████████████████████████████████████████████████████████████████| 808/808 [00:36<00:00, 22.38it/s]


In [11]:
#result_visulization
result_map=np.zeros((len(S2_layer_correct.flatten())))
result_map[np.asarray(pos)]=S2_layer_correct.flatten()[np.asarray(pos)]
result_map=result_map.reshape((height, width))
S2_layer_vis=np.where(S2_layer==np.inf, 0, S2_layer)

In [12]:
#local minima and maxima detection
h = 0.001
h_min = extrema.h_minima(S2_layer, h)
h_min_vis=np.where(h_min==1, 100, S2_layer_vis)

#some corrections
bg_map=flm_binary+inner_parts
bg_map=np.where(bg_map>0, 255, 0)

h_min_correct=np.where(bg_map==0, h_min, 0)
h_min_correct_vis=np.where(h_min_correct==1, 100, S2_layer_vis)

C:\Users\yusukekita\Anaconda3\lib\site-packages\skimage\morphology\extrema.py:262: RuntimeWarning: invalid value encountered in subtract
  residue_img = rec_img - image


Next step is minor layers extraction

In [13]:
#local minima and maxima detection
h = 0.1
minor_layers=np.where(minor_layers==np.inf, 0, minor_layers)
h_max = extrema.h_maxima(minor_layers, h)

#some modification
h_max_correct=np.where(result_map==0, h_max, 0)
result_map_minor=np.where(h_max_correct==1, 200, im_bg_sub_2)

Get MFA of S2 and minor layers by using above info. 

In [15]:
#extract MFA values of S2, and minor layers cell by cell, in the separate manner
h_min_correct=h_min_correct.flatten()
h_max_correct=h_max_correct.flatten()
label_map_resize=label_map_resize.flatten()
MFA_im=MFA_im.flatten()
angle_im=angle_im.flatten()

#set list
#S2
MFA_S2_cell_by_cell_list=[]
angle_S2_cell_by_cell_list=[]
MFA_S2_cell_by_cell_hist_list=[]
#minor layers
MFA_minor_cell_by_cell_list=[]
angle_minor_cell_by_cell_list=[]
MFA_minor_cell_by_cell_hist_list=[]
#whole azimuthal angle
angle_cell_by_cell_list=[]

#get index of each layer beforehand
index_S2=np.where(h_min_correct==1)[0]
index_minor=np.where(h_max_correct==1)[0]

for i in range(len(radial_file_result_list)):
    #set list
    #S2
    MFA_S2_cell_by_cell_file=[]
    angle_S2_cell_by_cell_file=[]
    MFA_S2_cell_by_cell_hist_file=[]
    #minor layers
    MFA_minor_cell_by_cell_file=[]
    angle_minor_cell_by_cell_file=[]
    MFA_minor_cell_by_cell_hist_file=[]
    #whole azimuthal angle
    angle_cell_by_cell_file=[]
    
    for j in tqdm(radial_file_result_list[i]):
        #get index of cell of cell index j
        index_cell=np.where(label_map_resize==j)[0]
        
        #get index of S2 and minor layers of cell index j
        index_cell_and_S2=list(set(list(index_cell)) & set(list(index_S2)))
        index_cell_and_minor=list(set(list(index_cell)) & set(list(index_minor)))
        
        #get MFA and azimuthal angle of cell index j
        #S2
        MFA_val_S2=MFA_im[index_cell_and_S2]
        angle_val_S2=angle_im[index_cell_and_S2]
        #minor layers
        MFA_val_minor=MFA_im[index_cell_and_minor]
        angle_val_minor=angle_im[index_cell_and_minor]
        #whole azimuthal angle
        angle_val_whole=angle_im[index_cell]
        
        #get MFA distribution of cell index j
        hist_S2, bins=np.histogram(MFA_val_S2, bins=20, range=(0.01, 60))
        hist_minor, bins=np.histogram(MFA_val_minor, bins=20, range=(0.01, 60))
        MFA_bins=[(bins[k]+bins[k+1])//2 for k in range(len(bins)-1)]
        
        #save results in radial file unit
        #S2
        MFA_S2_cell_by_cell_file.append(MFA_val_S2)
        angle_S2_cell_by_cell_file.append(angle_val_S2)
        MFA_S2_cell_by_cell_hist_file.append(hist_S2)
        #minor layers
        MFA_minor_cell_by_cell_file.append(MFA_val_minor)
        angle_minor_cell_by_cell_file.append(angle_val_minor)
        MFA_minor_cell_by_cell_hist_file.append(hist_minor)
        #wholse azimuthal angle
        angle_cell_by_cell_file.append(angle_val_whole)
        
    #save all results
    #S2
    MFA_S2_cell_by_cell_list.append(MFA_S2_cell_by_cell_file)
    angle_S2_cell_by_cell_list.append(angle_S2_cell_by_cell_file)
    MFA_S2_cell_by_cell_hist_list.append(MFA_S2_cell_by_cell_hist_file)
    #minor
    MFA_minor_cell_by_cell_list.append(MFA_minor_cell_by_cell_file)
    angle_minor_cell_by_cell_list.append(angle_minor_cell_by_cell_file)
    MFA_minor_cell_by_cell_hist_list.append(MFA_minor_cell_by_cell_hist_file)
    #whole aziumthal angle
    angle_cell_by_cell_list.append(angle_cell_by_cell_file)

#reshape
label_map_resize=label_map_resize.reshape((height, width))
MFA_im=MFA_im.reshape((height, width))
angle_im=angle_im.reshape((height, width))

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:03<00:00,  7.14it/s]


MFA of radial and tangential walls are obtained in the separate manner.

In [24]:
# pixels whose values are equal to 255 in flm_b_im are set to bg in MFA_im
label_map_resize=label_map_resize.flatten()
flm_binary=flm_binary.flatten()
MFA_im=MFA_im.flatten()
angle_im=angle_im.flatten()
nuclei_unique=np.unique(label_map_resize)

#extract MFA values of each cell by using cell labels
#array
MFA_S2_rad_list=[]
MFA_minor_rad_list=[]
MFA_S2_tan_list=[]
MFA_minor_tan_list=[]
#hist
MFA_S2_rad_hist=[]
MFA_minor_rad_hist=[]
MFA_S2_tan_hist=[]
MFA_minor_tan_hist=[]
#aizmuthal angle
azimuthal_angle_hist=[]
azimuthal_angle_hist_smooth=[]
azimuthal_angle_param_list=[]

#get index of each layer beforehand
index_S2=np.where(h_min_correct==1)[0]
index_minor=np.where(h_max_correct==1)[0]

for i in tqdm(range(len(radial_file_result_list))):
    #array
    MFA_S2_rad_temp=[]
    MFA_S2_tan_temp=[]
    MFA_minor_rad_temp=[]
    MFA_minor_tan_temp=[]
    #hist
    MFA_S2_rad_hist_temp=[]
    MFA_S2_tan_hist_temp=[]
    MFA_minor_rad_hist_temp=[]
    MFA_minor_tan_hist_temp=[]
    #azimuthal angle
    azimuthal_angle_hist_temp=[]
    azimuthal_angle_hist_smooth_temp=[]
    azimuthal_angle_param_temp=[]
    
    for j, ind in enumerate(radial_file_result_list[i]):
        cell_index=np.where((label_map_resize==ind)&(flm_binary==0))[0]
        MFA_cell=MFA_im[cell_index]
        angle_cell=angle_im[cell_index]
        
        #get istograms of azimuthal angle
        hist, bins_=np.histogram(angle_cell, bins=30, range=(0, 180))
        
        #save
        azimuthal_angle_hist_temp.append(hist)
        
        #smoothing
        window=3
        w = np.ones(window)/window
        hist = np.convolve(hist, w, mode='same')
        
        #get bins for azimuthal angle histograms
        angle_bins_=[(bins_[i]+bins_[i+1])//2 for i in range(len(bins_)-1)]

        #
        param_ini = [np.max(hist)*1.2, 45, 5]
        bounds_1 = [np.max(hist)*0.6, 30, 0]
        bounds_2 = [np.max(hist)*1.5, 60, 20]
        popt, pcov = curve_fit(gauss_func, angle_bins_, hist, p0=param_ini, bounds=(bounds_1, bounds_2))
        fitting = gauss_func(angle_bins_, popt[0], popt[1], popt[2])

        #tangential wall regions are defined as within 4*sigma (5% residual in Lorentz function)
        mu=popt[1]
        sigma=popt[2]
        region=[mu-3*sigma, mu+3*sigma]

        #get pixels defined as tangentiall walls
        index_tan=np.where((region[0] < angle_cell) & (angle_cell< region[1]))[0]
        index_rad=[i for i in range(len(cell_index)) if i not in index_tan]
        
        #get S2 and minor layer contributions of radial and tangential wall
        index_S2_tan=list(set(list(cell_index[index_tan])) & set(list(index_S2)))
        index_S2_rad=list(set(list(cell_index[index_rad])) & set(list(index_S2)))
        index_minor_tan=list(set(list(cell_index[index_tan])) & set(list(index_minor)))
        index_minor_rad=list(set(list(cell_index[index_rad])) & set(list(index_minor)))
        
        #result visualization
        # fitting result
        if i==2:
            #visualize result on binarized map
            zero_mat=np.zeros(len(label_map_resize))
            zero_mat[cell_index]=192
            zero_mat=zero_mat.reshape((height, width))
            zero_mat_copy=copy.deepcopy(zero_mat)

            #calculate region props for visualization
            props=measure.regionprops(np.uint8(zero_mat))
            cy_target, cx_target = props[0].centroid

            zero_mat=zero_mat.flatten()
            zero_mat_copy=zero_mat_copy.flatten()
            zero_mat[cell_index[index_rad]]=64
            zero_mat_copy[cell_index[index_rad]]=64
            zero_mat[index_S2_rad]=255
            zero_mat[index_S2_tan]=32
            zero_mat_copy[index_minor_rad]=255
            zero_mat_copy[index_minor_tan]=32
            zero_mat=zero_mat.reshape((height, width))
            zero_mat_copy=zero_mat_copy.reshape((height, width))

            #visualization
            fig, ax = plt.subplots(1, 3, figsize=(9, 3))
            ax[0].plot(angle_bins_, hist, alpha=0.5, color='m')
            ax[0].plot(angle_bins_,fitting, 'k')
            ax[0].set_xlabel("Azimuthal angle (degree)", size=12)
            ax[0].set_ylabel("Frequency", size=12)

            ax[1].imshow(zero_mat[int(cy_target-150):int(cy_target+150), 
                                int(cx_target-150):int(cx_target+150)], cmap="jet")
            ax[1].tick_params(labelbottom=False,
               labelleft=False,
               labelright=False,
               labeltop=False)
            ax[1].tick_params(bottom=False,
               left=False,
               right=False,
               top=False)
            
            ax[2].imshow(zero_mat_copy[int(cy_target-150):int(cy_target+150), 
                                int(cx_target-150):int(cx_target+150)], cmap="jet")
            ax[2].tick_params(labelbottom=False,
               labelleft=False,
               labelright=False,
               labeltop=False)
            ax[2].tick_params(bottom=False,
               left=False,
               right=False,
               top=False)
            
            plt.tight_layout()
            #plt.savefig(os.path.join(save_path, "Tangential_wall_separation_radial_file_No"+str(i)+"_cell_No"+str(j)
            #                         +"_cell_ind_"+str(ind)+".png"), dpi=600)
            #plt.show()
            plt.close()
        else:
            pass
        
        #get MFA distribution of cell index j
        MFA_S2_rad=MFA_im[index_S2_rad]
        MFA_S2_tan=MFA_im[index_S2_tan]
        MFA_minor_rad=MFA_im[index_minor_rad]
        MFA_minor_tan=MFA_im[index_minor_tan]
        
        hist_rad_S2, bins=np.histogram(MFA_S2_rad, bins=20, range=(0.01, 60))
        hist_tan_S2, bins=np.histogram(MFA_S2_tan, bins=20, range=(0.01, 60))
        hist_rad_minor, bins=np.histogram(MFA_S2_rad, bins=20, range=(0.01, 60))
        hist_tan_minor, bins=np.histogram(MFA_S2_tan, bins=20, range=(0.01, 60))
        
        #save result
        #array
        MFA_S2_rad_temp.append(MFA_S2_rad)
        MFA_S2_tan_temp.append(MFA_S2_tan)
        MFA_minor_rad_temp.append(MFA_minor_rad)
        MFA_minor_tan_temp.append(MFA_minor_tan)
        #hist
        MFA_S2_rad_hist_temp.append(hist_rad_S2)
        MFA_S2_tan_hist_temp.append(hist_tan_S2)
        MFA_minor_rad_hist_temp.append(hist_rad_minor)
        MFA_minor_tan_hist_temp.append(hist_tan_minor)
        #azimuthal angle
        azimuthal_angle_hist_smooth_temp.append(hist)
        azimuthal_angle_param_temp.append(mu)
    
    #array
    MFA_S2_rad_list.append(MFA_S2_rad_temp)
    MFA_S2_tan_list.append(MFA_S2_tan_temp)
    MFA_minor_rad_list.append(MFA_minor_rad_temp)
    MFA_minor_tan_list.append(MFA_minor_tan_temp)
    
    #hist
    MFA_S2_rad_hist.append(MFA_S2_rad_hist_temp)
    MFA_S2_tan_hist.append(MFA_S2_tan_hist_temp)
    MFA_minor_rad_hist.append(MFA_minor_rad_hist_temp)
    MFA_minor_tan_hist.append(MFA_minor_tan_hist_temp)
    
    #azimuthal angle
    azimuthal_angle_hist.append(azimuthal_angle_hist_temp)
    azimuthal_angle_hist_smooth.append(azimuthal_angle_hist_smooth_temp)
    azimuthal_angle_param_list.append(azimuthal_angle_param_temp)

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [02:56<00:00,  9.79s/it]


In [31]:
#save results as npz
#save results
np.savez(os.path.join(npz_path, "MFA_distributions_cell_by_cell_layer_by_layerx40.npz"), 
         MFA_S2=MFA_S2_cell_by_cell_list, angle_S2=angle_S2_cell_by_cell_list,
         MFA_S2_hist=MFA_S2_cell_by_cell_hist_list, 
         MFA_minor=MFA_minor_cell_by_cell_list, angle_minor=angle_minor_cell_by_cell_list,
         MFA_minor_hist=MFA_minor_cell_by_cell_hist_list, 
         MFA_S2_rad=MFA_S2_rad_list, MFA_S2_tan=MFA_S2_tan_list,
         MFA_S2_rad_hist=MFA_S2_rad_hist, MFA_S2_tan_hist=MFA_S2_tan_hist,
         MFA_minor_rad=MFA_minor_rad_list, MFA_minor_tan=MFA_minor_tan_list, 
         MFA_minor_rad_hist=MFA_minor_rad_hist, MFA_minor_tan_hist=MFA_minor_tan_hist,
         MFA_bins=MFA_bins, azimuthal_angle=azimuthal_angle_hist, azimuthal_angle_smooth=azimuthal_angle_hist_smooth,  
         azimuthal_angle_param=azimuthal_angle_param_list, angle_bins=angle_bins_, 
         azimuthal_angle_ori=angle_cell_by_cell_list)